# Notebook exemple

## 1 - Imports et données

In [ ]:
ALL_DATASET=False

In [ ]:
import json
import numpy as np
import pandas as pd
import requests

from langchain.chains import StuffDocumentsChain, RetrievalQA, LLMChain, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings, OllamaEmbeddings
from langchain.llms import Ollama, BaseLLM
from langchain.schema import Document, Generation, LLMResult
from langchain.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain_community.llms import OpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from pathlib import Path
from tqdm import tqdm
from glob import glob


In [ ]:
if ALL_DATASET:
    file="./10p_accords_publics_et_thematiques_240815.parquet"
    df=pd.read_parquet(file)
    df.loc[df["theme"].isna(),"theme"]="Autre, précisez"
    df_sample=df.sample(1000,random_state=42)
    df_sample.to_parquet("10p_accords_publics_et_thematiques_240815_sample_of_1000.parquet")
else:
    df_sample=pd.read_parquet("./10p_accords_publics_et_thematiques_240815_sample_of_1000.parquet")

In [ ]:
df_sample=df_sample[:10]

## 2 - Vectorisation des textes

In [ ]:
text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=3000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

model_kwargs = {'device': 'cuda'}
embedder = HuggingFaceEmbeddings(model_name="BAAI/bge-m3", model_kwargs=model_kwargs,show_progress=False)

In [ ]:
vector_store = Chroma(embedding_function=embedder, persist_directory="./chroma_db")
for index, row in tqdm(df_sample.iterrows(), total=len(df_sample)):
    text=df_sample.texte[index]
    texts = text_splitter.create_documents([text])
    i=0
    for t in texts:
        t.metadata["id"]=f"{index}_{i}"
        t.metadata["index"]=f"{index}"
        vector_store.add_documents([t])
        i+=1

## 3 - Inférence avec Ollama

In [ ]:
class LocalOllamaLLM(BaseLLM):
    api_url : str
    def _generate(self, prompt, stop):
        response = requests.post(f"{self.api_url}/api/generate", json={"model": "mistral-large", "prompt": str(prompt) })
        response.raise_for_status()
        response_text=''.join([json.loads(line)['response'] for line in response.text.splitlines()])
        generations=[]
        generations.append([Generation(text=response_text)])
        return LLMResult(generations=generations)


    def _llm_type(self):
        return "local"  

In [ ]:
llm = LocalOllamaLLM(api_url="http://127.0.0.1:11434")

system_prompt = (
    " Répondez à la question posée "
    " Utilisez le contexte (sélection des meilleurs paragraphes liés à la question) donné pour répondre à la question "
    " Si la réponse ne se trouve pas dans le contexte, répondez par 'Non'"
    " Contexte : {context}  "
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
def search_and_invoke_llm(vector_store,index,query,k=5):
    if k==0:
        print(f"bug with {index}")
        return None
    else:
        pass
    try:
        retriever=vector_store.as_retriever(
        search_kwargs={
                "k": k, 
                "filter": {'index': index}
            }
        )
        chain = create_retrieval_chain(retriever, question_answer_chain)
        result=chain.invoke({"input": query})
        return result
    except:
        search_and_invoke_llm(vector_store,index,query,k=k-1)
    return None

In [ ]:
THEMATIQUES={
    "accord_methode_penibilite":"Accords de méthode (pénibilité)",
"accord_methode_pse":"Accords de méthode (PSE)",
"amenagement_temps_travail":"Aménagement du temps de travail (modulation, annualisation, cycles)",
"autres":"Autre, précisez",
"autres_condition_travail":"Autres dispositions de conditions de travail (CHSCT, médecine du travail, politique générale de prévention)",
"autres_dispositions_duree":"Autres dispositions durée et aménagement du temps de travail ",
"autres_dispositions_egalite":"Autres dispositions Egalité professionnelle",
"autres_dispositions_emploi":"Autres dispositions emploi",
"calendrier_negociation":"Calendrier des négociations",
"classifications":"Classifications",
"commision_paritaire":"Commissions paritaires",
"cet":"Compte épargne temps",
"couverture_complementaire":"Couverture complémentaire santé - maladie",
"don_jour":"Dispositifs don de jour et jour de solidarité",
"distribution_actions_gratuites":"Distribution d'actions gratuites",
"droit_deconnexion":"Droit à la déconnexion et outils numériques",
"droit_syndical":"Droit syndical, IRP, expression des salariés",
"duree_collective_temps_travail":"Durée collective du temps de travail",
"egalite_salariale":"Egalité salariale F/H",
"election_pro":"Elections professionnelles, prorogations de mandat et vote électronique",
"evolution_prime":"Evolution des primes",
"evolution_salariale":"Evolution des salaires (augmentation, gel, diminution)",
"fin_conflit":"Fin de conflit",
"conges":"Fixation des congés (jours fériés, ponts, RTT)",
"forfait":"Forfaits (en heures, en jours)",
"formation_pro":"Formation professionnelle",
"gpec":"GPEC",
"heures_supp":"Heures supplémentaires (contingent, majoration)",
"indemnites":"Indemnités (dont kilométrique)",
"interessement":"Intéressement",
"mesure_age":"Mesures d'âge (seniors, contrat de génération...)",
"mobilite":"Mobilité (géographique, professionnelle - promotions)",
"diversite":"Non discrimination - Diversité",
"participation":"Participation",
"pee_peg":"PEE ou PEG",
"pei":"PEI",
"penibilite":"Pénibilité du travail (1% pénibilité, prévention, compensation/réparation)",
"perco_percoi":"PERCO et PERCOI",
"performance_collecte":"Performance collective (accord de compétitivité)",
"prevoyance_collective":"Prévoyance collective, autre que santé maladie",
"prime_partage_profit":"Prime de partage des profits",
"qvt":"QVT, conciliation vie personnelle/vie professionnelle",
"reprise_des_donnees":"Reprise des données",
"retraite_complementaire":"Retraite complémentaire - supplémentaire",
"rupture_conventionnelle_collective":"Rupture conventionnelle collective",
"stress_rps":"Stress, risques psycho-sociaux",
"supplement_participation":"Supplément de participation",
"supplement_interessement":"Supplément d'intéressement",
"systeme_prime":"Système de prime (autre qu'évolution)",
"système_de_remuneration":"Système de rémunération (autres qu'évolution)",
"teletravail":"Télétravail",
"travail_temps_partiel":"Travail à temps partiel",
"travail_nuit":"Travail de nuit",
"travail_dimanche":"Travail du dimanche",
"travailleurs_handicapes":"Travailleurs handicapés"}

In [ ]:
already_done={el.split("/")[1].split(".")[0] for el in glob("results/*.answer")}
new_dir = Path('results').mkdir(exist_ok=True)

list_of_df=[]
for index, row in df_sample.iterrows():
    dict_answer=dict()
    answer=""
    if index not in already_done:
        for (k,v) in THEMATIQUES.items():
            Q0=f"Oui ou non : est-ce qu'il y a un article sur : {v}?"
            if ans:=search_and_invoke_llm(vector_store,index,Q0,k=2):
                answer_txt=ans['answer']
                reponse=0
                if answer_txt.lower().startswith("oui") :
                    reponse=1
                dict_answer[k]=reponse
                answer_k = f"{k} : {answer_txt}"
                answer += answer_k
            answer += "\n-----\n"
            
        if answer:
            with open(f"results/{index}.answer","w") as f:
                f.write(answer)
        list_of_df.append(pd.DataFrame(dict_answer, index=[index]))

In [ ]:
df_results=pd.concat(list_of_df)